<a href="https://colab.research.google.com/github/hemanths03/CS6910_Assignment_2/blob/main/Part_B_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing_WandB

In [1]:
!pip install wandb
import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 5.0 MB/s 
     |████████████████████████████████| 144 kB 45.7 MB/s 
     |████████████████████████████████| 181 kB 55.2 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=214fc138cecaa7316953dd46932d46c84a9b02f87bb6cdd52aa7e61430bb46e6
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#Importing the Required Libraries

In [2]:
#Importing some Required Libraries
import wandb
import numpy as np
from wandb.keras import WandbCallback
from tensorflow import keras
import random
from math import ceil

#Importing the Models
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import Xception

#For Data generation
from keras.preprocessing.image import ImageDataGenerator

#Import the required models
from tensorflow.keras.models import Sequential, Model

#Import the required layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten, Dropout, BatchNormalization

#Import some other Libraries
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

#WandB Login



In [ ]:
!wandb login

#Get the Dataset

In [ ]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip

#unzip the Dataset

In [ ]:
!unzip 'nature_12K.zip'

In [ ]:
import tensorflow as tf

#Load train and test paths

In [ ]:
train = 'inaturalist_12K/train'
test = 'inaturalist_12K/val'

#Setting Up the paramters of input image

In [ ]:
#Height Width and Depth of input image
(H , W , D ) = (224 , 224 , 3)

#number of labels in the dataset
n_labels = 10

#batch size
batch_size = 128

#Data Generation

In [ ]:
#Data generator
data_gen = ImageDataGenerator(rescale=1./255,validation_split = 0.1)

In [ ]:
#Generating the train_set (training data)
train_data = data_gen.flow_from_directory(
                                          directory = train,
                                          color_mode = 'rgb',       
                                          shuffle = True,
                                          seed = 10,
                                          target_size = (H , W),
                                          batch_size = batch_size,
                                          subset = 'training'
                                          )

In [ ]:
#Generating the val_set (validation data)
val_data = data_gen.flow_from_directory(
                                        directory = train,
                                        color_mode = 'rgb',
                                        shuffle = True,
                                        seed = 10,
                                        target_size = (H , W), 
                                        batch_size = batch_size,
                                        subset = 'validation'
                                        )

In [ ]:
#Generating the test_set (testing data)
test_data = data_gen.flow_from_directory(
                                        directory = test,
                                        color_mode = 'rgb',
                                        shuffle = False,
                                        seed = 10,
                                        target_size = (H , W), 
                                        batch_size = batch_size
                                        )

#Sweep Configuration

In [ ]:
sweep_config_temp = {
  "name": "sweep_1",
  "method": "grid",
  "early_terminate" : {
      "type": "hyperband",
      "min_iter": 3
  },
  "metric": {
      "name": "accuracy",
      "goal": "maximize"
  },
  "parameters": {
        "model": {
            "values": ["inception_v3", "inception_resnet_v2", "resnet_50", "xception"]
        },
        "set_conv_layers" : {
            "values" : ["all" , "none" , 1 , 2]
        },
      "num_neurons" : {
          "values" : [1024] , 512]
      }
    }
}

#Function Helper returns the required model

In [ ]:
def Helper(model , H , W , D):
    if model == "inception_v3":
        return InceptionV3(input_shape = (H , W , D), weights = "imagenet", include_top = False)
    elif model == "inception_resnet_v2":
        return InceptionResNetV2(input_shape = (H , W , D), weights = "imagenet", include_top = False)
    elif model == "resnet_50":
        return ResNet50(input_shape = (H , W , D), weights = "imagenet", include_top = False)
    elif model == "xception":
        return Xception(input_shape = (H , W , D), weights = "imagenet", include_top = False)

In [ ]:
from keras.models import Model
from keras.layers import Dense , Flatten

In [ ]:
def train():
    
    #Initializing wandb
    run = wandb.init()
    config = wandb.config

    #Setting the run Name
    run.name = "model_" + str(config.model) + "_set_conv_layers_" + str(config.set_conv_layers)

    #Getting the first last and second last convolution layers in the model
    if(config.model == "inception_v3"):
      l2_conv_layers = [12, 17]
    elif(config.model == "inception_resnet_v2"):
      l2_conv_layers = [2, 4]
    elif(config.model == "resnet_50"):
      l2_conv_layers = [4, 7]
    elif(config.model == "xception"):
      l2_conv_layers = [3, 6]

    
    #getting the required model
    model = Helper(config.model , H , W , D)
    
    model.trainable = False
    
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(config.num_neurons , activation = "relu")(flat1)
    output = Dense(10 , activation = "softmax")(class1)
    
    
    model = Model(inputs = model.inputs , outputs = output)
    

    if (config.set_conv_layers == "none"): #Traning none of the layers
      print("")
    else:
      if(config.set_conv_layers == "all"): #Traning all the layers
        model.trainable = True 
      else:
        for layer in model.layers[- l2_conv_layers[config.set_conv_layers - 1] : ]: #Training from first last or second last convolution layers
          layer.trainable = True

    #Compiling the model with Adam as the optimizer
    model.compile(optimizer = Adam(1e-5),
                        loss = 'categorical_crossentropy',
                        metrics = ['accuracy']
                        )
    #fitting the model on train data
    model.fit(train_data,
                    steps_per_epoch = ceil((float) (train_data.n) / train_data.batch_size),
                    epochs = 10,
                    callbacks = [WandbCallback()],
                    validation_data = val_data,
                    validation_steps = ceil((float) (val_data.n) / val_data.batch_size)
                    )
      
    run.finish()

#Setting the sweep id for the sweep

In [ ]:
sweep_id = wandb.sweep(sweep_config_temp, entity="cs21m027_cs21m011", project="DL_ASG_2_final_Part_B")

In [ ]:
wandb.agent(sweep_id, train)

#Paramaters

In [ ]:
#"values": ["inception_v3", "inception_resnet_v2", "resnet_50", "xception"]
#"values" : ["all" , "none" , 1 , 2]

#Checking the numbers of first last and second last layers of all models

In [ ]:
#inception_model = InceptionV3(input_shape = (100 , 100 , 3), weights = "imagenet", include_top = False)

In [ ]:
# check the layers by name
#for i,layer in enumerate(inception_model.layers):
#    print(i,layer.name)

In [ ]:
##img = '/content/inaturalist_12K/train'

In [ ]:
#paths = train_data.filepaths

In [ ]:
#import cv2

In [ ]:
'''for i in range(1000):
  img = paths[i]
  image = cv2.imread(img)
  print(image.shape)'''

In [ ]:
#image.shape